# Development Notebook

### Imports

In [9]:
# -- public imports

import os
import pandas as pd
from brat_parser import get_entities_relations_attributes_groups

In [10]:
# -- private import

In [11]:
# -- dev imports

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Importing the dataset

In [12]:
base_path = '../../data/UCL/dataset2'
path = os.path.join(base_path, 'ArgumentAnnotatedEssays-2.0')
text_file_dir = os.path.join(path, 'brat-project-final')

texts = []
annotated_texts = []
for file in os.listdir(text_file_dir):
    essay_num, file_extension = file.split('.')
    if file_extension == 'ann':
        with open(os.path.join(text_file_dir, file), 'r') as f:
            df_temp = pd.read_csv(f, delimiter='\t', header=None, names=['label_type', 'label', 'text'])
            df_temp[['label', 'label_comp1', 'label_comp2']] = df_temp.label.str.split(expand=True)
            df_temp['doc_id'] = essay_num
            annotated_texts.append(df_temp)
    elif file_extension == 'txt':
        with open(os.path.join(text_file_dir, file), 'r') as f:
            texts.append((essay_num, f.read()))
    else:
        continue

df_texts = pd.DataFrame.from_records(texts, columns=['doc_id', 'text'])
df_annotated = pd.concat(annotated_texts)

assert sorted(df_annotated.doc_id.unique()) == sorted(df_texts.doc_id)

In [13]:
df_annotated

,label_type,label,text,label_comp1,label_comp2,doc_id
0,T1,MajorClaim,alternative means of transportation and intern...,358,464,essay031
1,T2,MajorClaim,alternative forms of transport and internation...,2393,2565,essay031
2,T3,Claim,some people claim the convenience of automobile,2325,2372,essay031
3,A1,Stance,NaN,T3,Against,essay031
4,T4,Claim,it is crucial to alter automobiles to the othe...,822,933,essay031
...,...,...,...,...,...,...
21,T13,Premise,"In order to promote the country's attractions,...",1468,1595,essay028
22,T14,Premise,Only well planned and well targeted advertisin...,1597,1692,essay028
23,R5,supports,NaN,Arg1:T12,Arg2:T11,essay028
24,R6,supports,NaN,Arg1:T13,Arg2:T11,essay028


In [14]:
df_texts.to_json(
    os.path.join(base_path, 'df_texts.json')
)

df_annotated.reset_index(drop=True).to_json(
    os.path.join(base_path, 'df_full_annotated.json')
)

In [15]:
ids_argument_segment = df_annotated.label_type.str.startswith('T')

df_arguments = df_annotated[ids_argument_segment]

df_arguments

,label_type,label,text,label_comp1,label_comp2,doc_id
0,T1,MajorClaim,alternative means of transportation and intern...,358,464,essay031
1,T2,MajorClaim,alternative forms of transport and internation...,2393,2565,essay031
2,T3,Claim,some people claim the convenience of automobile,2325,2372,essay031
4,T4,Claim,it is crucial to alter automobiles to the othe...,822,933,essay031
6,T5,Premise,an increase in the number of automobiles bring...,561,638,essay031
...,...,...,...,...,...,...
16,T10,Premise,"Thanks to ads we learn about new products, we ...",1209,1305,essay028
18,T11,Claim,"These days, not only many businesses, but also...",1307,1393,essay028
20,T12,Premise,tourism makes up one-third of the Czech Republ...,1408,1466,essay028
21,T13,Premise,"In order to promote the country's attractions,...",1468,1595,essay028


In [16]:
df_arguments = df_arguments.rename(columns={'label_comp1':'span_start', 'label_comp2':'span_end'}).astype(
    {'span_start': int, 'span_end':int}
)
df_arguments.dtypes

label_type    object
label         object
text          object
span_start     int64
span_end       int64
doc_id        object
dtype: object

In [17]:
records = []
df_arguments = df_arguments.sort_values(['doc_id', 'span_start', 'span_end'])
for (doc_id, text) in df_texts.sort_values('doc_id').itertuples(index=False):
    df_argument = df_arguments[df_arguments.doc_id == doc_id]
    prev_span = 0
    for i, (text_segment, span_start, span_end) in enumerate(df_argument[['text', 'span_start', 'span_end']].itertuples(index=False)):
        try:
            assert text_segment == text[span_start: span_end]
        except Exception as e:
            print(f'{text_segment}\n{text[span_start: span_end]}\n')
        # all the exception were manually checked. These are because of qutoe chars, this is a hot fix.!!!! TODO
            df_arguments['text'][df_arguments.text == text_segment] = text[span_start: span_end]
        records.append(('O', 'Other', text[prev_span: span_start], prev_span, span_start, doc_id))
        prev_span = span_end
    records.append(('O', 'Other', text[prev_span:], prev_span, len(text), doc_id))

murdering criminals is therefore immoral and hard to accept
"murdering" criminals is therefore immoral and hard to accept



/var/folders/_6/scw_50_57tb8mmbpq6ylnl6w0000gn/T/ipykernel_14116/2945358557.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_arguments['text'][df_arguments.text == text_segment] = text[span_start: span_end]
/var/folders/_6/scw_50_57tb8mmbpq6ylnl6w0000gn/T/ipykernel_14116/2945358557.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_arguments['text'][df_arguments.text == text_segment] = text[span_start: span_end]
/var/folders/_6/scw_50_57tb8mmbpq6ylnl6w0000gn/T/ipykernel_14116/2945358557.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the 

Click is a very interesting comedy, with a serious approach about the importance of having a balanced life between family and work businesses
"Click" is a very interesting comedy, with a serious approach about the importance of having a balanced life between family and work businesses

Blood diamond, an adaptation of a real story in South Africa, focuses on the link between diamonds and conflict
"Blood diamond", an adaptation of a real story in South Africa, focuses on the link between diamonds and conflict

rush hours are usually not the direct consequence of drivers
"rush hours" are usually not the direct consequence of drivers



/var/folders/_6/scw_50_57tb8mmbpq6ylnl6w0000gn/T/ipykernel_14116/2945358557.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_arguments['text'][df_arguments.text == text_segment] = text[span_start: span_end]


In [18]:
df_other = pd.DataFrame.from_records(records, columns=df_arguments.columns)
df_other

,label_type,label,text,span_start,span_end,doc_id
0,O,Other,Should students be taught to compete or to coo...,0,503,essay001
1,O,Other,".\nFirst of all,",575,591,essay001
2,O,Other,.,714,716,essay001
3,O,Other,.,851,853,essay001
4,O,Other,.,1086,1088,essay001
...,...,...,...,...,...,...
6486,O,Other,.,1339,1341,essay402
6487,O,Other,", so",1388,1393,essay402
6488,O,Other,". Secondly,",1436,1448,essay402
6489,O,Other,". \nIn conclusion,",1525,1543,essay402


In [26]:
df_combined = pd.concat([df_other, df_arguments])
df_combined = df_combined.sort_values(['doc_id', 'span_start', 'span_end']).reset_index(drop=True)
df_combined[df_combined.doc_id == 'essay001']

,label_type,label,text,span_start,span_end,doc_id
0,O,Other,Should students be taught to compete or to coo...,0,503,essay001
1,T1,MajorClaim,we should attach more importance to cooperatio...,503,575,essay001
2,O,Other,".\nFirst of all,",575,591,essay001
3,T3,Claim,"through cooperation, children can learn about ...",591,714,essay001
4,O,Other,.,714,716,essay001
5,T4,Premise,What we acquired from team work is not only ho...,716,851,essay001
6,O,Other,.,851,853,essay001
7,T5,Premise,"During the process of cooperation, children ca...",853,1086,essay001
8,O,Other,.,1086,1088,essay001
9,T6,Premise,All of these skills help them to get on well w...,1088,1191,essay001


In [27]:
df_combined.to_json(
        os.path.join(base_path, 'df_with_other.json')
)

In [35]:
df_essay001 = df_combined[df_combined.doc_id == 'essay001']
''.join(df_essay001.text)

"Should students be taught to compete or to cooperate?\n\nIt is always said that competition can effectively promote the development of economy. In order to survive in the competition, companies continue to improve their products and service, and as a result, the whole society prospers. However, when we discuss the issue of competition or cooperation, what we are concerned about is not the whole society, but the development of an individual's whole life. From this point of view, I firmly believe that we should attach more importance to cooperation during primary education.\nFirst of all, through cooperation, children can learn about interpersonal skills which are significant in the future life of all students. What we acquired from team work is not only how to achieve the same goal with others but more importantly, how to get along with others. During the process of cooperation, children can learn about how to listen to opinions of others, how to communicate with others, how to think c

In [33]:
df_essay001.values[1]

array(['T1', 'MajorClaim',
       'we should attach more importance to cooperation during primary education',
       503, 575, 'essay001'], dtype=object)

In [34]:
df_essay001.values[2]

array(['O', 'Other', '.\nFirst of all, ', 575, 591, 'essay001'],
      dtype=object)

In [27]:
from argminer.data import DataProcessor, TUDarmstadtProcessor

In [78]:
test = DataProcessor(1)
test.preprocess()
test.pr()

AttributeError: 'DataProcessor' object has no attribute '_preprocess'

In [88]:
path = '../../data/UCL/dataset2/ArgumentAnnotatedEssays-2.0/brat-project-final'
test2 = TUDarmstadtProcessor(path)
test2.preprocess()
test2.process('bio')

Found non-matching segments:--------------------------------------------------

murdering criminals is therefore immoral and hard to accept

"murdering" criminals is therefore immoral and hard to accept

Found non-matching segments:--------------------------------------------------

Click is a very interesting comedy, with a serious approach about the importance of having a balanced life between family and work businesses

"Click" is a very interesting comedy, with a serious approach about the importance of having a balanced life between family and work businesses

Found non-matching segments:--------------------------------------------------

Blood diamond, an adaptation of a real story in South Africa, focuses on the link between diamonds and conflict

"Blood diamond", an adaptation of a real story in South Africa, focuses on the link between diamonds and conflict

Found non-matching segments:--------------------------------------------------

rush hours are usually not the direct co

In [85]:
df = pd.DataFrame({'a':[1,2,3]})

df2 = df
df2.a = df2.a.apply(lambda x: x*2)
print(df)

   a
0  2
1  4
2  6


[autoreload of argminer.data failed: Traceback (most recent call last):
  File "/Users/yousefnami/opt/anaconda3/envs/argument-mining/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/Users/yousefnami/opt/anaconda3/envs/argument-mining/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/Users/yousefnami/opt/anaconda3/envs/argument-mining/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 839, in exec_module
  File "<frozen importlib._bootstrap_external>", line 976, in get_code
  File "<frozen importlib._bootstrap_external>", line 906, in source_to_code
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Users/yousefnami/Desktop/Main/0.Educatio

In [66]:
test2.preprocess()

preprocessed


Exception: Preprocess method has already been called.